In [ ]:
# ==================== Import modules & initialize DB ==================== #

import app_database as db
import importlib                                            # To reload the modules
importlib.reload(db)                                        # Reload the db module
import os
import datetime

db.drop_all_tables()

#* -------------------- Log in and Sessions -------------------- #

db.create_table(
    'users',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"                    # Internal unique ID (Primary Key)
    "username TEXT UNIQUE,"                                    # Unique username
    "password TEXT,"                                           # Password (should be hashed)
    "email TEXT UNIQUE,"                                       # Unique email address
    "SSO_id INTEGER UNIQUE,"                                   # External SSO authentication ID
    "registration_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP"    # Account registration date
)

list_of_users = (
    ["admin", "admin", "admin@gevernova.com", 101010, datetime.datetime.now()],
    ["user1", "user1", "user1@gevernova.com", 111111, datetime.datetime.now()],
    ["user2", "user2", "user2@gevernova.com", 222222, datetime.datetime.now()],
    ["user3", "user3", "user3@gevernova.com", 333333, datetime.datetime.now()],
)
for data in list_of_users:
    db.add_element_to_table('users', data)

db.create_table(
    'users_sessions',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"                       # Session ID (Primary Key)
    "user_id INTEGER REFERENCES users(id) ON DELETE CASCADE,"     # Linked user ID (Foreign Key)
    "session_token TEXT UNIQUE,"                                  # Unique session token
    "is_active BOOLEAN DEFAULT true,"                             # Session status (active/inactive)
    "session_start TIMESTAMP DEFAULT CURRENT_TIMESTAMP,"          # Session creation timestamp
    "last_activity TIMESTAMP"                                     # Last user activity timestamp
)

list_of_sessions = (
    [101010, "tokenAdmin", True, datetime.datetime.now(), datetime.datetime.now()],
    [111111, "token1", True, datetime.datetime.now(), datetime.datetime.now()],
    [222222, "token2", True, datetime.datetime.now(), datetime.datetime.now()],
    [333333, "token3", True, datetime.datetime.now(), datetime.datetime.now()],
)
for data in list_of_sessions:
    db.add_element_to_table('users_sessions', data)


db.create_table(
    'login_history',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"                      # Event ID (Primary Key)
    "user_id INTEGER REFERENCES users(id),"                      # User who logged in (Foreign Key)
    "login_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,"            # Login timestamp
    "logout_time TIMESTAMP"                                      # Logout timestamp (nullable)
)

list_of_logins = (
    [101010, datetime.datetime.now()-datetime.timedelta(days=1), datetime.datetime.now()],
    [111111, datetime.datetime.now()-datetime.timedelta(days=1), datetime.datetime.now()],
    [222222, datetime.datetime.now()-datetime.timedelta(days=1), datetime.datetime.now()],
    [333333, datetime.datetime.now()-datetime.timedelta(days=1), datetime.datetime.now()],
)
for data in list_of_logins:
    db.add_element_to_table('login_history', data)


#* -------------------- Projects -------------------- #

db.create_table(
    'projects',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"                      # Project ID (Primary Key)
    "user_id INTEGER REFERENCES users(id) ON DELETE CASCADE,"    # Project creator (Foreign Key)
    "title TEXT,"                                                # Project title
    "description TEXT,"                                          # Detailed project description
    "status TEXT,"                                               # Project status (e.g. Open, In Progress, Closed)
    "creation_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,"         # Date of project creation
    "last_update TIMESTAMP DEFAULT CURRENT_TIMESTAMP"            # Last project update timestamp
)

list_of_projects = (
    [1, "Project 1", "Description of the project 1", "Open", datetime.datetime.now(), datetime.datetime.now()],
    [2, "Project 2", "Description of the project 2", "Open", datetime.datetime.now(), datetime.datetime.now()],
    [3, "Project 3", "Description of the project 3", "Open", datetime.datetime.now(), datetime.datetime.now()],
    [4, "Project 4", "Description of the project 4", "Open", datetime.datetime.now(), datetime.datetime.now()],
)
for data in list_of_projects:
    db.add_element_to_table('projects', data)


db.create_table(
    'comments',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"                             # Comment ID (Primary Key)
    "user_id INTEGER REFERENCES users(id) ON DELETE CASCADE,"           # Author of the comment (FK)
    "project_id INTEGER REFERENCES projects(id) ON DELETE CASCADE,"     # Target project (FK)
    "content TEXT,"                                                     # Comment text/content
    "creation_date TEXT"                                                # Timestamp of comment creation
)

list_of_comments = (
    [1, 1, "Comment 1", datetime.datetime.now()],
    [2, 2, "Comment 2", datetime.datetime.now()],
    [3, 3, "Comment 3", datetime.datetime.now()],
    [4, 4, "Comment 4", datetime.datetime.now()],
)
for data in list_of_comments:
    db.add_element_to_table('comments', data)


db.create_table(
    'projects_members',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"                             # Membership ID (Primary Key)
    "project_id INTEGER REFERENCES projects(id) ON DELETE CASCADE,"     # Related project (FK)
    "user_id INTEGER REFERENCES users(id) ON DELETE CASCADE,"           # Member user (FK)
    "role TEXT,"                                                        # Role in the project (e.g. admin, contributor)
    "add_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP"                      # When the user was added to the project
)

list_of_projects_members = (
    [1, 1, "ApporteLesCroissants", datetime.datetime.now()],
    [2, 2, "ParleTrop", datetime.datetime.now()],
    [3, 3, "TheBoss", datetime.datetime.now()],
    [4, 4, "Nothing", datetime.datetime.now()],
)
for data in list_of_projects_members:
    db.add_element_to_table('projects_members', data)

#* -------------------- Products and Orders -------------------- #

db.create_table(
    'products',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"     # Product ID (Primary Key)
    "reference INTEGER,"                        # Internal product reference code
    "name TEXT,"                                # Product name
    "description TEXT"                          # Product description
)

list_of_products = (
    [10000, "Banana", "Fruit"],
    [11111, "Apple", "Fruit"],
    [22222, "Orange", "Fruit"],
    [33333, "Tomato", "Vegetable"],
    [44444, "Carot", "Vegetable"],
    [55555, "Cucumber", "Vegetable"],
    [66666, "Chicken", "Meat"],
    [77777, "Fish", "Seafood"],
)
for data in list_of_products:
    db.add_element_to_table('products', data)

db.create_table(
    'orders',
    "id INTEGER PRIMARY KEY AUTOINCREMENT,"     # Order ID (Primary Key)
    "user_id INTEGER,"                          # Customer (FK, ideally should reference users(id))
    "product_id INTEGER,"                       # Product ordered (FK, ideally should reference products(id))
    "quantity INTEGER,"                         # Quantity ordered
    "order_date TEXT"                           # Date of the order
)

# ----- Function to read the instructions from a file ----- #
#? nothing to do here

# products is the table that contains the products

In [2]:
# Display the content of all the tables
db.print_all_tables()


----- Table: sqlite_sequence -----


,name,seq
0,users,4
1,users_sessions,4
2,login_history,4
3,projects,4
4,comments,4
5,projects_members,4
6,products,8



----- Table: users -----


,id,username,password,email,SSO_id,registration_date
0,1,admin,admin,admin@gevernova.com,101010,2025-03-13_10:57:16.267049
1,2,user1,user1,user1@gevernova.com,111111,2025-03-13_10:57:16.267049
2,3,user2,user2,user2@gevernova.com,222222,2025-03-13_10:57:16.267049
3,4,user3,user3,user3@gevernova.com,333333,2025-03-13_10:57:16.267049



----- Table: users_sessions -----


...

,id,project_id,user_id,role,add_date
0,1,1,1,ApporteLesCroissants,2025-03-13_10:57:16.543622
1,2,2,2,ParleTrop,2025-03-13_10:57:16.543622
2,3,3,3,TheBoss,2025-03-13_10:57:16.543622
3,4,4,4,Nothing,2025-03-13_10:57:16.543622



----- Table: products -----


,id,reference,name,description
0,1,10000,Banana,Fruit
1,2,11111,Apple,Fruit
2,3,22222,Orange,Fruit
3,4,33333,Tomato,Vegetable
4,5,44444,Carot,Vegetable
5,6,55555,Cucumber,Vegetable
6,7,66666,Chicken,Meat
7,8,77777,Fish,Seafood



----- Table: orders -----


,id,user_id,product_id,quantity,order_date
